In [1]:
# Dependencies and Setup
import json
import pandas as pd
import requests
import numpy as py
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, inspect, func, distinct
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3

In [2]:
from sqlite3 import Error

In [3]:
engine = create_engine("sqlite:///../SqLIte_files/NBAfantasyML.sqlite")
conn = engine.connect()

In [4]:
# create function to try and connect to a sqlite database.  if it doesn't work, create one...
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()

In [5]:
if __name__ == '__main__':
    create_connection("../SqLIte_files/NBAfantasyML.sqlite")

2.6.0


In [6]:
#Bring in data from internet as object then convert ot Json
player_response_obj=requests.get("http://data.nba.net/prod/v1/2018/players.json")
player_response=player_response_obj.json()

In [7]:
# Count # of Players
player_response_count=player_response["league"]["standard"]
player_count=len(player_response_count)
player_count

498

In [8]:
#Bring in elements for NBA_Player Table
NBA_Players=[]

for i in range (0,player_count-1):
    player_response_list=player_response["league"]["standard"][i]
    firstName=player_response_list["firstName"]
    lastName=player_response_list["lastName"]
    personId=player_response_list["personId"]
    teamId=player_response_list["teamId"]
    heightFeet=player_response_list["heightFeet"]
    heightInches=player_response_list["heightInches"]
    weightPounds=player_response_list["weightPounds"]
    dateOfBirthUTC=player_response_list["dateOfBirthUTC"]
    collegeName=player_response_list["collegeName"]

    # Create an object list to store NBA Player data as an object
   
    NBA_Players.append({"firstName":firstName,
                       "lastName":lastName,
                       "personId":personId,
                       "teamId":teamId,
                       "heightFeet":heightFeet,
                       "heightInches":heightInches,
                       "weightPounds":weightPounds,
                       "dateOfBirth":dateOfBirthUTC,
                       "collegeName":collegeName})

    #Test Object list
    NBA_Players

In [9]:
NBA_Player_df_initial=pd.DataFrame(NBA_Players)
NBA_Player_df=NBA_Player_df_initial[["personId","lastName","firstName","teamId",
                                     "heightFeet","heightInches","weightPounds",
                                     "dateOfBirth","collegeName"]]
NBA_Player_df

,personId,lastName,firstName,teamId,heightFeet,heightInches,weightPounds,dateOfBirth,collegeName
0,203518,Abrines,Alex,1610612760,6,6,200,1993-08-01,
1,203112,Acy,Quincy,1610612756,6,7,240,1990-10-06,Baylor
2,1629121,Adams,Jaylen,1610612737,6,2,190,1996-05-04,St. Bonaventure
3,203500,Adams,Steven,1610612760,7,0,265,1993-07-20,Pittsburgh
4,1628389,Adebayo,Bam,1610612748,6,10,255,1997-07-18,Kentucky
5,1629061,Adel,Deng,1610612739,6,7,200,1997-02-01,
6,200746,Aldridge,LaMarcus,1610612759,6,11,260,1985-07-19,Texas
7,1628959,Alkins,Rawle,1610612741,6,5,225,1997-10-29,
8,1628960,Allen,Grayson,1610612762,6,5,198,1995-10-08,Duke
9,1628386,Allen,Jarrett,1610612751,6,11,237,1998-04-21,Texas


In [10]:
# Load Player non Stat data
NBA_Player_df.to_sql('PLAYER_NON_STAT',conn, if_exists='replace', index=False)

OperationalError: (sqlite3.OperationalError) too many SQL variables [SQL: 'INSERT INTO "PLAYER_NON_STAT" ("personId", "lastName", "firstName", "teamId", "heightFeet", "heightInches", "weightPounds", "dateOfBirth", "collegeName") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, 